In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Importing the Data

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import json
import string
import re
# import nltk
# nltk.download()  # This is used if you do not have nltk and its files downloaded on your computer.
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

In [ ]:
testList = []

with open('../input/dataset/dataFiles.txt', 'r') as filehandle:
    testList = json.load(filehandle)
    
len(testList)

In [ ]:
def findWords(lst, keywords, radius, dropwords, usewords):
    porter = PorterStemmer()
    drop = [porter.stem(word) for word in dropwords]
    use = [porter.stem(word) for word in usewords]
    keys = [porter.stem(word) for word in keywords]
    
    for word in drop:
        if word in lst:
            return []

    for word in use:
        if word not in lst:
            return []
    
    poses = []
    for i in range(len(lst)):
        if lst[i] in keys:
            poses.append(i)

#     print('[8/9] Getting Shorter List ...')
    words = []
    dist = radius
    for i in range(len(poses)):
        startPos = -1
        endPos = -1

        if i > 0:
            if poses[i] - poses[i-1] > dist:
                startPos = poses[i] - dist
            else:
                startPos = poses[i] - int((poses[i] - poses[i-1])/2)
        else:
            if poses[i] - dist >= 0:
                startPos = poses[i] - dist
            else:
                startPos = 0

        if i < len(poses)-1:
            if poses[i+1] - poses[i] > dist:
                endPos = poses[i] + dist
            else:
                endPos = poses[i] + int((poses[i+1] - poses[i])/2)
        else:
            if poses[i] + dist < len(lst):
                endPos = poses[i] + dist
            else:
                endPos = len(lst)-1

    #     print('[{}/{}] ...'.format(i, len(poses)))
    #     print(startPos, poses[i], endPos)
        words = words + lst[startPos:endPos]
    #     print(myWords[startPos:endPos])

#     print('[9/9] Done!')

    for word in words:
        if word in use or word in drop:
            words.remove(word)
    
    return words

In [ ]:
wordsFound = []
keyWords = ['factors', 'symptoms', 'risk', 'risks', 'symptom', 'factor']
# keyWords = ['symptoms']
radius = 20
removeDocs = ['dengue']
useDocs = ['covid', 'sars', 'coronavirus']

for i in range(len(testList)):
    if i % 1000 == 0:
        print('[{}/{}]'.format(i, len(testList)))
        
    wordsFound = wordsFound + findWords(testList[i], keyWords, radius, removeDocs, useDocs)

# Counts the number of times a word is used.
wordCounts = {}
for word in wordsFound:
    if word in wordCounts:
        wordCounts[word] = wordCounts[word] + 1
    else:
        wordCounts[word] = 1
        
wordCounts = sorted(wordCounts.items(), key=lambda x: x[1], reverse=True)

wordCounts = [i for i in wordCounts if i[0] not in keyWords]
import copy
longlist = copy.deepcopy(wordCounts)
wordCounts[0:20]

In [ ]:
plt.figure()
plt.bar([i[0] for i in wordCounts[0:20]], [i[1] for i in wordCounts[0:20]])
plt.title('Word Counts with Radius {}'.format(radius))
plt.xlabel('Words')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.show()

In [ ]:
rfs = ['pulmonari', 'smoke', 'coinfect', 'socioeconom', 'pregnant', 'behavior', 'infant', 'preexist' ]
for thing in longlist:
    for key in rfs:
        if thing[0] == key:
            print(thing[0] + " " + str(thing[1]))
# print(longlist)

In [ ]:
for rfsword in rfs:
    wordsFound = []
    keyWords = [rfsword]
    # keyWords = ['symptoms']
    radius = 20
    removeDocs = ['dengue']
    useDocs = ['covid', 'sars', 'coronavirus']

    for i in range(len(testList)):
        if i % 10000 == 0:
            print('[{}/{}]'.format(i, len(testList)))

        wordsFound = wordsFound + findWords(testList[i], keyWords, radius, removeDocs, useDocs)

    # Counts the number of times a word is used.
    wordCounts = {}
    for word in wordsFound:
        if word in wordCounts:
            wordCounts[word] = wordCounts[word] + 1
        else:
            wordCounts[word] = 1

    wordCounts = sorted(wordCounts.items(), key=lambda x: x[1], reverse=True)

    wordCounts = [i for i in wordCounts if i[0] not in keyWords]
    wordCounts[0:20]
    plt.figure()
    plt.bar([i[0] for i in wordCounts[0:20]], [i[1] for i in wordCounts[0:20]])
    plt.title(rfsword + ' Word Counts with Radius {}'.format(radius))
    plt.xlabel('Words')
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.show()